# Setup

In [1]:
import pandas as pd
import sagemaker

from pathlib import Path

In [2]:
input_path = Path('/root/kaggle-projects/tabular-playground-march-21/input')
s3_project_dir = 's3://briorep-s3-ds/Aaron-temp/Projects-Personal/kaggle-tabular-playground-march-21/'

In [3]:
# Read test dataset
pd_test = pd.read_csv(input_path / 'test.csv', index_col='id')
display(pd_test.head(1))

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
id,,,,,,,,,,,,,,,,,,,,,
5,A,F,A,A,F,BI,A,AH,AX,A,...,0.73569,0.578366,0.723154,0.228037,0.356227,0.551249,0.655693,0.598331,0.359987,0.947489


In [4]:
# Read submission dataset
pd_submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')
display(pd_submission.head(1))

,target
id,
5,0.5


# Preprocess Data

In [5]:
# SageMaker parameters
image_uri='246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-sklearn-automl:0.2-1-cpu-py3'
training_job_name='kaggle-tab-dpp2-1-93ef8f32aa2a4cacac7741c3d3610561282cba3699c84'
test_data = s3_project_dir + 'test_no_index.csv'

In [12]:
# Create Estimator
from sagemaker.estimator import Estimator
preprocessor_estimator = Estimator(
    image_uri=image_uri,
    role='arn:aws:iam::169327718394:role/bri-sagemaker',
    instance_count=1,
    instance_type='ml.m5.2xlarge')
preprocessor_estimator = preprocessor_estimator.attach(
    training_job_name=training_job_name)


2021-03-15 17:05:56 Starting - Preparing the instances for training
2021-03-15 17:05:56 Downloading - Downloading input data
2021-03-15 17:05:56 Training - Training image download completed. Training in progress.
2021-03-15 17:05:56 Uploading - Uploading generated training model
2021-03-15 17:05:56 Completed - Training job completed


In [13]:
# Create Transformer
preprocessor_transformer = preprocessor_estimator.transformer(
    instance_count=1,
    instance_type='ml.m5.2xlarge',
     accept='text/csv',
    assemble_with='Line',
    strategy='MultiRecord',
    max_payload=0,  # no limit
    output_path=s3_project_dir + 'test_no_index_processed.csv',
    output_kms_key=None,
    env = {'SAGEMAKER_MODEL_SERVER_TIMEOUT' : '3600'})

In [14]:
# Preprocess data
preprocessor_transformer.transform(data=test_data, wait=False)

# Model Inference 

In [ ]:
image_uri='174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:latest'
training_job_name='tuning-job-1-b40c384659414ba195-005-f4c7aa0a'

In [43]:
from sagemaker.sklearn.estimator import SKLearn

In [45]:
#preprocessor = SKLearn.attach('kaggle-tab-dpp2-1-93ef8f32aa2a4cacac7741c3d3610561282cba3699c84')

In [46]:
# Create transformer from SageMaker Training Job
sagemaker.estimator.EstimatorBase()

TypeError: Can't instantiate abstract class EstimatorBase with abstract methods create_model, hyperparameters, training_image_uri

In [38]:
# Create transformer from SageMaker Model
model_preprocessor = sagemaker.model.Model(
    image_uri='246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-sklearn-automl:0.2-1-cpu-py3',
    model_data='s3://briorep-s3-ds/Aaron-temp/Projects-Personal/kaggle-tabular-playground-march-21/kaggle-tabular-march-21/data-processor-models/kaggle-tab-dpp2-1-93ef8f32aa2a4cacac7741c3d3610561282cba3699c84/output/model.tar.gz'
)

# transformer_preprocessor = model_preprocessor.transformer(
#     instance_count=1,
#     instance_type='ml.m5.4xlarge',
#     output_path='s3://briorep-s3-ds/Aaron-temp/Projects-Personal/kaggle-tabular-playground-march-21/kaggle-tabular-march-21/test/'
# )

# Predictions

In [ ]:
#submission['target'] = clf.predict_proba(pd_test)[:, 1]
#submission.to_csv('random_forest.csv')